In [1]:
import nltk
import spacy
import pandas as pd
from tqdm import tqdm
import os
import nltk
import numpy as np
from pattern.text.en import singularize
import nltk
from nltk.corpus import stopwords
stop_words = stopwords.words('english')


# Checking for sentence classification

In [50]:
folder = "../LitCovid_Treatment_json/"

dirs = os.listdir(folder)

for file in tqdm(dirs, desc = "dirs"):
    try: 
        if file not in ["exceptionlist.txt", "label2id.pkl", "predict.tf_record"]:
            file_path = os.path.join(folder, file)
            temp = pd.read_json(file_path)
            sections = [temp.sentences.values[i]["Section"] for i in range(temp.shape[0])]
            if any(s != "METHODS" for s in sections):
                print(file)
                print(sections)
    except:
        print("Error lol")
        print(file)

dirs: 100%|████████████████████████████████████████████████████████████████████████| 6459/6459 [01:44<00:00, 61.84it/s]

Error lol
exceptionlist.txt
Error lol
label2id.pkl
Error lol
predict.tf_record


In [3]:
custom_syns = [['COVID-19', "SARS-CoV-2", "SARS-CoV-2 infection", "COVID-19 infection", 
  "coronavirus disease 2019 ( COVID-19 )", 'Covid-19', 'laboratory-confirmed COVID-19', 
  "coronavirus disease 2019", "COVID-19 disease", "coronavirus disease", "SARS-CoV", 
  "COVID-19 positive", "SARS-COV-2", "SARS-CoV-2 infections",  "Covid-19 infection", 
  "COVID-19 infected", "COVID-19-positive", "SARS-CoV-2 infected", "SARS-CoV-2-infected",
  "2019-nCoV", "COVID-19 infections", "COVID 19", "COVID19", "Covid-19 infection",
  "severe acute respiratory syndrome coronavirus 2 ( SARS-CoV-2 ) infection", 
  'coronavirus disease ( COVID-19 )', "CoV infections", "coronavirus", "2019-new coronavirus",
  "SARS-CoV-2 virus", "coronavirus infections"],
 ["severe COVID-19", "severe and critical COVID-19", "severe COVID-19 infection", 
  'severe coronavirus disease 2019 ( COVID-19 )',
  'severe coronavirus disease 2019 ( COVID-19 )', 
  'severe acute respiratory syndrome ( SARS )', 'severe and critical COVID-19',
  'severe acute respiratory syndrome coronavirus 2',
  'critically ill COVID-19', "severe COVID-19 disease"],
 ["severe COVID-19 pneumonia", "severe pneumonia", "COVID-19 pneumonia", "SARS-CoV-2 pneumonia",
  'novel coronavirus pneumonia', "2019-nCoV pneumonia"],
 ["male", "men", "males", "male sex"],
 ["infection", "infected"],
 ["female", "women", "females", "female sex"],
 ["hospitalized", "critically ill", "severe disease", "critical", "severe cases",
  "critical illness", "severely ill", "severe illness"],
 ["comorbidity", "comorbidities"],
 ["ICU", "intensive care unit ( ICU )", "intensive care"],
 ["diabetes", "diabetes mellitus", "diabetic"],
 ["elderly", "older age", "older adults", "older"],
 ["hydroxychloroquine", "chloroquine"],
 ["dead", "deceased"]]

lower_syns = []
for syn in custom_syns:
    lower_syns.append([s.lower() for s in syn])

    
def get_syn(ent):
    covid_syns = ["covid", "sars-cov", "corona", "ncov", "cov infections"]
    severe_syns = ["severe", "critical", "hospitalized"]
    if any(word in ent.lower() for word in covid_syns) and any(word in ent.lower() for word in severe_syns):
        return "severe covid-19"
    elif any(word in ent.lower() for word in covid_syns) and "pneumonia" in ent.lower():
        return "COVID-19 pneumonia"
    elif any(word in ent.lower() for word in severe_syns):
        return "severe"
    else:
        for l in lower_syns:
            if ent.lower() in l:
                return l[0]
        
    return ent

# Get data

In [4]:

folder = "../LitCovid_Treatment_json/"
dirs = os.listdir(folder)
pmids = []
entities = []
singulars = 0
for file in tqdm(dirs, desc = "dirs"):
# for file in dirs[2:4]:
    if file not in ["exceptionlist.txt", "label2id.pkl", "predict.tf_record"]:
        
        #get filepath and read in data
        file_path = os.path.join(folder, file)
        temp = pd.read_json(file_path)
        pmid = file[:-5]
        
        for i in range(temp.shape[0]):
            
            #get entities sentence by sentence
            ents_dict = temp.sentences.values[i].get("entities")
            if len(ents_dict.keys()) > 0:
                temp_ents = [(ents_dict[k]["text"], ents_dict[k]["class"], ents_dict[k]["negation"], pmid) for k in ents_dict.keys()]
                true_ents = []
                for i in range(len(temp_ents)):
                    ent = temp_ents[i][0]
                    cl = temp_ents[i][1]
                    neg = temp_ents[i][2]
                    added = False
                    
                    if len(ent) > 1: #make sure it's not punctuation
       
                        if len(ent.split()) > 1: #if phrase
                            
                            #check to see if it's in custom set of synonyms
                            new_ent = get_syn(ent)
                            true_ents.append((new_ent, cl, neg, pmid))
                        
                        else: #if single word
                            if ent.lower() not in stop_words: #if not stop word
                                
                                #check to see if it's in custom set of synonyms
                                new_ent = get_syn(ent)
                                
                                if new_ent != ent:
                                    true_ents.append((new_ent, cl, neg, pmid))
                                  
                                else:
                                    ent = singularize(ent)
                                    new_ent = get_syn(ent)
                                    true_ents.append((new_ent, cl, neg, pmid))       
                                    
                entities += true_ents


dirs: 100%|████████████████████████████████████████████████████████████████████████| 6459/6459 [01:36<00:00, 67.25it/s]


In [5]:
ent_df = pd.DataFrame(entities, columns=["text", "type", "negation", "pmid"])
ent_df.to_csv("LitCovid_treatment_ents.csv")
ent_df


,text,type,negation,pmid
0,covid-19,Participant,0,31981224
1,relevant,Participant,0,31981224
2,host defense against,Outcome,0,31981224
3,covid-19,Participant,0,31981224
4,covid-19,Participant,0,31996494
5,inflammation,Outcome,0,32013309
6,covid-19,Participant,0,32013309
7,inhibited,modifier,0,32013309
8,accelerate,modifier,0,32015560
9,control,Outcome,0,32015560


# Building weighted graph

In [2]:
from collections import Counter
import itertools as it

# get data and group
ent_df = pd.read_csv("LitCovid_treatment_ents.csv")
g = ent_df.groupby(["pmid", "type"])

#initialize variables
groups = list(g.groups)
types = ["Participant", "Outcome", "Intervention"]
edge_weight = Counter()
nodes = []

for p in tqdm(ent_df.pmid.values[:10]):
    temp = []
    for tup in list(it.product([p], types)):
        if tup in groups:
            
            #get pmid entities
            grp = g.get_group(tup)
            
            #format appropriately
            temp_nodes = ["{} ({})".format(t, tup[1][0]) for t in grp.text.values]
            temp.append(temp_nodes)
            nodes += temp_nodes
        else:
            temp.append([])
            
    #get edges
    products = [list(it.product(temp[0], temp[1])), list(it.product(temp[0], temp[2])), list(it.product(temp[1], temp[2]))]
    
    #add edges to adj list
    for prod in products:
        if prod != []:
            edge_weight.update(prod)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 70.02it/s]


In [21]:
edge_weight

Counter({('covid-19 (P)', 'host defense against (O)'): 8,
         ('relevant (P)', 'host defense against (O)'): 4,
         ('covid-19 (P)', 'inflammation (O)'): 3})

In [20]:
import networkx as nx
from pyvis.network import Network
import matplotlib.pyplot as plt
#same intervention/population -> different outcomes, etc.
#categorize segments by intervention/population
#independence amongst categories
#what are treatments with same outcome
#what are possible outcomes
#build graph from adj list
G = nx.Graph()
for item in edge_weight.items():
    G.add_edge(item[0][0], item[0][1], weight=item[1])

#only show edge if enough hits
# cutoff = sorted(list(edge_weight.values()))[-2]
elarge = [(u, v) for (u, v, d) in G.edges(data=True) if d["weight"] >= cutoff]
esmall = [(u, v) for (u, v, d) in G.edges(data=True) if d["weight"] <= 0.5]

g = Network(height=800, width=800, notebook=True)
g.barnes_hut()
g.from_nx(G)
g.show("ex.html")


(('covid-19 (P)', 'host defense against (O)'), 8)
(('relevant (P)', 'host defense against (O)'), 4)
(('covid-19 (P)', 'inflammation (O)'), 3)


In [25]:
with open('edges.pickle', 'rb') as file:
    edge_weight = pickle.load(file)

In [31]:
[edge for edge in edge_weight.most_common() if "(P)" in edge[0][0] and "(I)" in edge[0][1]]

[(('covid-19 (P)', 'covid-19 (I)'), 101549),
 (('covid-19 (P)', 'hydroxychloroquine (I)'), 33819),
 (('severe (P)', 'covid-19 (I)'), 30392),
 (('severe (P)', 'severe (I)'), 22645),
 (('covid-19 (P)', 'control (I)'), 22079),
 (('age (P)', 'covid-19 (I)'), 20083),
 (('covid-19 (P)', 'severe (I)'), 19893),
 (('male (P)', 'covid-19 (I)'), 14340),
 (('severe covid-19 (P)', 'covid-19 (I)'), 12404),
 (('covid-19 (P)', 'HCQ (I)'), 11633),
 (('diabetes (P)', 'covid-19 (I)'), 10910),
 (('diabetes (P)', 'diabetes (I)'), 9238),
 (('covid-19 (P)', 'convalescent plasma (I)'), 8783),
 (('severe covid-19 (P)', 'control (I)'), 8260),
 (('severe covid-19 (P)', 'tocilizumab (I)'), 8204),
 (('severe (P)', 'hydroxychloroquine (I)'), 7898),
 (('elderly (P)', 'covid-19 (I)'), 7776),
 (('covid-19 (P)', 'remdesivir (I)'), 7639),
 (('covid-19 (P)', 'tocilizumab (I)'), 7615),
 (('female (P)', 'covid-19 (I)'), 7108),
 (('covid-19 (P)', 'placebo (I)'), 6943),
 (('covid-19 (P)', 'corticosteroid (I)'), 6870),
 (('se

In [53]:
temp = ent_df[((ent_df.type == "Participant") & (ent_df.text == "covid-19")) |
       ((ent_df.type == "Intervention") & (ent_df.text == "hydroxychloroquine"))]
grps = temp.groupby("pmid")
grps.describe()

Unnamed: 0                                                           \
              count           mean        std       min        25%       50%   
pmid                                                                           
31981224        2.0       1.500000   2.121320       0.0       0.75       1.5   
31996494        1.0       4.000000        NaN       4.0       4.00       4.0   
32013309        1.0       6.000000        NaN       6.0       6.00       6.0   
32026148        2.0      70.500000   0.707107      70.0      70.25      70.5   
32034659        1.0      90.000000        NaN      90.0      90.00      90.0   
32052466        1.0     158.000000        NaN     158.0     158.00     158.0   
32074550        1.0     286.000000        NaN     286.0     286.00     286.0   
32075365        4.0     321.250000   6.652067     313.0     318.25     321.5   
32079150        1.0     421.000000        NaN     421.0     421.00     421.0   
32081636        1.0     440.000000        NaN     440.0     440.00     440.0   
32096367        2.0     634.000000   2.828427     632.0     633.00     634.0   
32096395        1.0     659.000000        NaN     659.0     659.00     659.0   
32096396        1.0     661.000000        NaN     661.0     661.00     661.0   
32104907        1.0     692.000000        NaN     692.0     692.00     692.0   
32106567        1.0     786.000000        NaN     786.0     786.00     786.0   
32108862        1.0     819.000000        NaN     819.0     819.00     819.0   
32115733        2.0     852.500000   0.707107     852.0     852.25     852.5   
32117569        1.0     859.000000        NaN     859.0     859.00     859.0   
32118639        3.0     941.000000   4.000000     937.0     939.00     941.0   
32125140        1.0    1094.000000        NaN    1094.0    1094.00    1094.0   
32125362        2.0    1129.500000   4.949747    1126.0    1127.75    1129.5   
32129518        2.0    1146.000000   1.414214    1145.0    1145.50    1146.0   
32134278        1.0    1191.000000        NaN    1191.0    1191.00    1191.0   
32145714        4.0    1252.000000  17.204651    1230.0    1246.50    1253.0   
32147496        1.0    1276.000000        NaN    1276.0    1276.00    1276.0   
32147628        1.0    1279.000000        NaN    1279.0    1279.00    1279.0   
32147731        1.0    1280.000000        NaN    1280.0    1280.00    1280.0   
32149769        2.0    1315.000000   2.828427    1313.0    1314.00    1315.0   
32150618       12.0    1366.166667  10.726631    1347.0    1359.75    1369.5   
32152082        1.0    1386.000000        NaN    1386.0    1386.00    1386.0   
...             ...            ...        ...       ...        ...       ...   
32773926        1.0  120727.000000        NaN  120727.0  120727.00  120727.0   
32774170        1.0  120742.000000        NaN  120742.0  120742.00  120742.0   
32774508        4.0  120757.000000   1.825742  120755.0  120755.75  120757.0   
32774918        3.0  120773.333333   1.527525  120772.0  120772.50  120773.0   
32774998        2.0  120789.500000   2.121320  120788.0  120788.75  120789.5   
32775090        1.0  120813.000000        NaN  120813.0  120813.00  120813.0   
32775778        3.0  120844.666667   8.736895  120835.0  120841.00  120847.0   
32775814        2.0  120878.500000   6.363961  120874.0  120876.25  120878.5   
32775837        1.0  120916.000000        NaN  120916.0  120916.00  120916.0   
32775986        5.0  120939.800000   5.674504  120932.0  120936.00  120942.0   
32776022        2.0  120970.000000   1.414214  120969.0  120969.50  120970.0   
32776139        1.0  120973.000000        NaN  120973.0  120973.00  120973.0   
32776161        2.0  120999.500000   0.707107  120999.0  120999.25  120999.5   
32776197        2.0  121030.000000  11.313708  121022.0  121026.00  121030.0   
32776298        1.0  121093.000000        NaN  121093.0  121093.00  121093.0   
32776309        1.0  121150.000000        NaN  121150.0  121150.00  121150.0   
32776354 

In [55]:
grps.get_group(31981224)

,Unnamed: 0,text,type,negation,pmid
0,0,covid-19,Participant,0,31981224
3,3,covid-19,Participant,0,31981224


In [57]:
tuples = list(zip(*[ent_df['pmid'], ent_df['Unnamed: 0']]))
index = pd.MultiIndex.from_tuples(tuples, names=['pmid', 'orig_index'])

In [59]:
temp_ent_df = ent_df.copy()
temp_ent_df.index = index
temp_ent_df

Unnamed: 0                         text          type  \
pmid     orig_index                                                          
31981224 0                    0                     covid-19   Participant   
         1                    1                     relevant   Participant   
         2                    2         host defense against       Outcome   
         3                    3                     covid-19   Participant   
31996494 4                    4                     covid-19   Participant   
32013309 5                    5                 inflammation       Outcome   
         6                    6                     covid-19   Participant   
         7                    7                    inhibited      modifier   
32015560 8                    8                   accelerate      modifier   
         9                    9                      control       Outcome   
32017984 10                  10          2019-nCoV infection   Participant   
         11                  11                           26       measure   
         12                  12                        death       Outcome   
         13                  13                     covid-19       Outcome   
32022276 14                  14                   Persistent      modifier   
         15                  15                       H-bond       Outcome   
         16                  16                 salt bridges       Outcome   
         17                  17                      epitope  Intervention   
         18                  18                        MHC-I  Intervention   
         19                  19            2019-nCoV surface   Participant   
         20                  20                  interaction       Outcome   
         21                  21                          CTL  Intervention   
         22                  22                          CTL       Outcome   
         23                  23                 sequential B       Outcome   
         24                  24                 CTL epitopes  Intervention   
         25                  25                          CTL       Outcome   
         26                  26                        eight       measure   
         27                  27                       B cell       Outcome   
         28                  28                Epitope-MHC-I       Outcome   
32022370 29                  29                   remdesivir  Intervention   
...                         ...                          ...           ...   
32777784 121639          121639            peak liver injury       Outcome   
         121640          121640                   detectable      modifier   
         121641          121641            Mild liver injury   Participant   
         121642          121642                        worse      modifier   
         121643          121643                      outcome       Outcome   
         121644          121644             liver impairment       Outcome   
         121645          121645                       severe       Outcome   
         121646          121646                   associated      modifier   
         121647          121647       multiple-organ failure       Outcome   
         121648          121648        acute vascular events       Outcome   
         121649          121649   prevalence of liver injury       Outcome   
         121650          121650                       severe  Intervention   
         121651          121651                    mortality       Outcome   
         121652          121652                     covid-19  Intervention   
         121653          121653                 liver injury       Outcome   
         121654          121654                     covid-19   Participant   
         121655          121655                     elevated   Participant   
         121656          121656  alanine aminotransferase or   Participant   
         121657          121657            

# Frequencies of PICO element text

In [126]:
from collections import Counter
p_count = Counter(ent_df[ent_df.type == "Participant"].text.values)
i_count = Counter(ent_df[ent_df.type == "Intervention"].text.values)
o_count = Counter(ent_df[ent_df.type == "Outcome"].text.values)

In [24]:
p_count.most_common(100).items

NameError: name 'p_count' is not defined